# Bayesian Logistic regression with PyMC

In [ ]:
%%manim --disable_caching -qm -v WARNING SigmoidSurfacesLast_one

class SigmoidSurfacesLast_one(ThreeDScene):
    def construct(self):


        X_new = X[::15]
        y_new = y[::15]
        # Define the ranges based on your data
        x0_min, x0_max = X_new[:, 0].min(), X_new[:, 0].max()
        x1_min, x1_max = X_new[:, 1].min(), X_new[:, 1].max()
        
        # Create the 3D axes
        axes = ThreeDAxes(
            x_range=[x0_min, x0_max],
            y_range=[x1_min, x1_max],
            z_range=[-0.2, 1.2],
            x_length=7,
            y_length=7,
            z_length=5,
        )
        self.add(axes)
        
        # Define the sigmoid function
        def sigmoid(x, y, alpha, beta0, beta1):
            return 1 / (1 + np.exp(-(alpha + beta0 * x + beta1 * y)))
        
        num_samples = 20  # Adjust as needed
        indices = np.random.choice(len(alpha_samples), size=num_samples, replace=False)

        for idx in indices:
            a = alpha_samples[idx]
            b0 = beta_samples0[idx]
            b1 = beta_samples1[idx]

            surface = Surface(
                lambda u, v: axes.c2p(
                    u,
                    v,
                    sigmoid(u, v, a, b0, b1)
                ),
                u_range=[x0_min, x0_max],
                v_range=[x1_min, x1_max],
                resolution=(15, 15),
                fill_opacity=0.2,
                stroke_color=BLACK,
                checkerboard_colors= False,
                
            )
            self.add(surface)
            surface.set_color(GRAY)
        # Compute the mean parameters
        mean_alpha = np.mean(alpha_samples)
        mean_beta0 = np.mean(beta_samples0)
        mean_beta1 = np.mean(beta_samples1)

        # Create the mean sigmoid surface
        mean_surface = Surface(
            lambda u, v: axes.c2p(
                u,
                v,
                sigmoid(u, v, mean_alpha, mean_beta0, mean_beta1)
            ),
            u_range=[x0_min, x0_max],
            v_range=[x1_min, x1_max],
            resolution=(25, 25),
            fill_opacity=0.9,
            checkerboard_colors= False,
        )
        self.add(mean_surface)
        mean_surface.set_color(MAROON) 
        
        # Add data points to the scene
        data_points = VGroup()
        for i in range(len(X_new)):
            dot = Dot3D(
                point=axes.c2p(X_new[i, 0], X_new[i, 1], y_new[i]),
                radius=0.05,
                color=BLACK,
            )
            data_points.add(dot)
        self.add(data_points)
        
        # Set initial camera orientation
        self.set_camera_orientation(phi=45 * DEGREES, theta=-45 * DEGREES)

        
        # Start ambient camera rotation
        self.begin_ambient_camera_rotation(rate=0.5)
        self.wait(17)  # Adjust the duration as needed
        
        # Stop the camera rotation
        self.stop_ambient_camera_rotation()


# Bayesian Linear regression with pymc

In [ ]:
%%manim --disable_caching -ql -v WARNING LinearSurfaces40

class LinearSurfaces40(ThreeDScene):
    def construct(self):


        # Subsample data for plotting
        X_new = X[::5]
        y_new = y[::5]

        # Define the ranges based on your data
        x0_min, x0_max = X_new[:, 0].min(), X_new[:, 0].max()
        x1_min, x1_max = X_new[:, 1].min(), X_new[:, 1].max()
        
        # Compute the mean parameters
        mean_alpha = np.mean(intercept)
        mean_beta0 = np.mean(slopes0)
        mean_beta1 = np.mean(slopes1)

        # Define the Linear model function
        def linear_model(x, y, alpha, beta0, beta1):
            return alpha + beta0 * x + beta1 * y

        # Compute z-values at the corners of the x-y plane using mean parameters
        z_corners = [
            linear_model(x0_min, x1_min, mean_alpha, mean_beta0, mean_beta1),
            linear_model(x0_min, x1_max, mean_alpha, mean_beta0, mean_beta1),
            linear_model(x0_max, x1_min, mean_alpha, mean_beta0, mean_beta1),
            linear_model(x0_max, x1_max, mean_alpha, mean_beta0, mean_beta1),
        ]

        # Compute z_min and z_max considering both data and model outputs
        z_min = min(y_new.min(), min(z_corners)) - 3  # Subtract buffer if needed
        z_max = max(y_new.max(), max(z_corners)) + 3  # Add buffer if needed

        # Create the 3D axes with the correct z_range and include numbers
        axes = ThreeDAxes(
            x_range=[x0_min - 3, x0_max + 3, (x0_max + 3 - (x0_min - 3)) / 5],
            y_range=[x1_min - 3, x1_max + 3, (x1_max + 3 - (x1_min - 3)) / 5],
            z_range=[z_min, z_max, (z_max - z_min) / 5],
            x_length=7,
            y_length=7,
            z_length=5,
        )
        self.add(axes)

        # Add numbers to the axes
        x_numbers = VGroup()
        y_numbers = VGroup()
        z_numbers = VGroup()

        # Get the ticks positions
        z_ticks = np.arange(0,8,1).astype(int)

        # Create labels for z-axis
        for z in z_ticks:
            z_label = Text(f"{z:.1f}", font_size=24)
            z_label.move_to(axes.c2p(0, 0, z))
            z_label.rotate(PI / 2, axis=UP)
            z_label.shift(LEFT * 0.1)
            #z_label.always_face_camera(self.camera)
            z_numbers.add(z_label)

        # Add labels to scene
        self.add(axes, x_numbers, y_numbers, z_numbers)

        num_samples = 36  # Total number of samples
        group_size = 9    # Size of each group

        # We'll use the first num_samples indices
        indices = np.arange(0, 7, 1).astype(int)

        # Split indices into groups of group_size
        groups = [indices[i:i + group_size] for i in range(0, num_samples, group_size)]

        # Plot the mean regression plane for each group
        for idx, group in enumerate(groups):
            a_mean = np.mean(intercept[group])
            b0_mean = np.mean(slopes0[group])
            b1_mean = np.mean(slopes1[group])

            surface = Surface(
                lambda u, v: axes.c2p(
                    u,
                    v,
                    linear_model(u, v, a_mean, b0_mean, b1_mean)
                ),
                u_range=[x0_min, x0_max],
                v_range=[x1_min, x1_max],
                resolution=(15, 15),
                fill_opacity=0.2,
                color=BLUE,
            )
            self.add(surface)

        # Create the overall mean linear model surface
        mean_surface = Surface(
            lambda u, v: axes.c2p(
                u,
                v,
                linear_model(u, v, mean_alpha, mean_beta0, mean_beta1)
            ),
            u_range=[x0_min, x0_max],
            v_range=[x1_min, x1_max],
            resolution=(25, 25),
            fill_opacity=0.7,
            stroke_opacity=0.5,
            color=MAROON,
        )
        self.add(mean_surface)

        # Add data points to the scene
        data_points = VGroup()
        for i in range(len(X_new)):
            dot = Dot3D(
                point=axes.c2p(X_new[i, 0], X_new[i, 1], y_new[i]),
                radius=0.05,
                color=BLACK,
            )
            data_points.add(dot)
        self.add(data_points)

        # Set initial camera orientation
        self.set_camera_orientation(phi=45 * DEGREES, theta=-45 * DEGREES, zoom=1.2)

        # Start ambient camera rotation
        self.begin_ambient_camera_rotation(rate=0.5)
        self.wait(1)  # Adjust the duration as needed

        # Stop the camera rotation
        self.stop_ambient_camera_rotation()


# NN

In [ ]:
%%manim -qm -v WARNING NeuralNetworkSceneLastMB


class NeuralNetworkSceneLastMB(Scene):

    def construct(self):
        # Create layers without layer labels
        input_layer = self.create_layer(3, "x")
        hidden_layer = self.create_layer(4, "h")
        output_layer = self.create_layer(2, "z")
        Intro_text = Text("MB", font_size=24, weight= BOLD,color= BLUE_D).to_edge(UP).shift(LEFT*6.05)
        self.add(Intro_text)
        Neural_net_text = Text("Neural Network", font_size=16, weight= BOLD,color= BLUE_E).next_to(Intro_text,DOWN,buff=0.2) #.shift(RIGHT*0.1)
        self.add(Neural_net_text)

        # Position layers
        layers = VGroup(input_layer, hidden_layer, output_layer)
        layers.arrange(RIGHT, buff=2.7)
        self.add(layers)
        self.layers = layers

        # Create layer labels
        input_label = Text("Input Layer", font_size=24)
        hidden_label = Text("Hidden Layer", font_size=24)
        output_label = Text("Output Layer", font_size=24)
        layer_labels = VGroup(input_label, hidden_label, output_label)

        # Position layer labels above the layers at the same height
        for label, layer in zip(layer_labels, layers):
            label.move_to(layer.get_center())
            label.shift(UP * 3.2)  

        self.add(layer_labels)
        self.layer_labels = layer_labels

        # Draw connections with weights
        connections_ih, weights_ih,weights_ih_after = self.connect_layers(input_layer, hidden_layer, "w")
        connections_oh, weights_ho,weights_oh_after = self.connect_layers(hidden_layer, output_layer, "v")
        self.connections_oh = connections_oh
        self.connections_ih = connections_ih

        # Combine all connections and weights
        connections = connections_ih + connections_oh
        weights = weights_ih + weights_ho
        weights_after = weights_ih_after + weights_oh_after
        self.weights_after = weights_after

        # Add connections and weight labels to the scene
        self.add(connections, weights)

        # Animate forward pass with computations
        self.forward_pass(input_layer, hidden_layer, output_layer, connections, weights)

        # Animate backpropagation with derivatives
        self.backpropagation(input_layer, hidden_layer, output_layer, connections, weights)

    def animate_wave(self, connections, color, direction='forward', run_time=1):
 
        dots = VGroup()
        #lines = VGroup() #optional
        animations = []
        
        for connection in connections:
            # Reverse the connection path if direction is 'backward'
            if direction == 'backward':
                path = Line(
                    connection.get_end(),
                    connection.get_start(),
                    stroke_color=connection.get_stroke_color(),
                    stroke_width=connection.get_stroke_width()
                )
            else:
                path = connection.copy()
            
            # Create a dot at the start of the path
            dot = Dot(color=color, radius=0.05)
            dot.move_to(path.get_start())
            dots.add(dot)
            
            # Animate the dot moving along the path
            animation = MoveAlongPath(dot, path, run_time=run_time, rate_func=linear)
            animations.append(animation)
        
        
        #self.add(lines) #optional
        #self.remove(lines) #optional
        self.add(dots)
        self.play(*animations, lag_ratio=0)
        self.remove(dots)
        

    


    def create_layer(self, num_neurons, label_prefix):
        """
        Creates a layer with a specified number of neurons and labels each neuron uniquely.
        """
        neurons = VGroup()
        neuron_labels = VGroup()
        for i in range(num_neurons):
            neuron = Circle(radius=0.3, color=BLUE, fill_opacity=0)
            neurons.add(neuron)
        neurons.arrange(DOWN, buff=1)  # Increased buff for more vertical spacing
        for i, neuron in enumerate(neurons):
            # Create unique labels like x₁, x₂, etc.
            label = MathTex(f"{label_prefix}_{{{i+1}}}")
            label.scale(0.6)
            label.next_to(neuron, UP, buff=0.05)  # if we want to change the label to be inside the circle, change LEFT to UP
            neuron_labels.add(label)
        layer = VGroup(neurons, neuron_labels)
        return layer  # Return layer without layer label

    def connect_layers(self, layer1, layer2, weight_prefix):

        connections = VGroup()
        weight_labels = VGroup()
        weight_labels_after = VGroup()
        neurons1 = layer1[0]  
        neurons2 = layer2[0]
        for i, neuron1 in enumerate(neurons1):
            for j, neuron2 in enumerate(neurons2):
                connection = Line(
                    neuron1.get_right(),
                    neuron2.get_left(),
                    stroke_color=GREY,
                    stroke_width=2
                )
                connections.add(connection)
                weight_label = MathTex(f"{weight_prefix}_{{{j+1}{i+1}}}")
                weight_label_new = MathTex(f"\\tilde {weight_prefix}_{{{j+1}{i+1}}}",font_size=16)
                weight_label.scale(0.5)
                midpoint = connection.get_midpoint()
                direction = neuron2.get_left() - neuron1.get_right()
                unit_direction = direction / np.linalg.norm(direction)
                perp_direction = np.array([-unit_direction[1], unit_direction[0], 0])
                weight_label.move_to(midpoint + perp_direction * 0.5)
                weight_labels.add(weight_label)
                weight_label_new.move_to(midpoint + perp_direction * 0.5)
                weight_labels_after.add(weight_label_new)

        return connections, weight_labels,weight_labels_after

    def forward_pass(self, input_layer, hidden_layer, output_layer, connections, weights):

        forward_pass = Text("Forward Pass", font_size=30)
        forward_pass.to_corner(DOWN + LEFT)
        SIGMOID = MathTex("\\sigma(x) = \\frac{1}{1 + e^{-x}}",font_size=20)
        Bias = MathTex("b_j, c_t, \\quad j \\in [1,4], t \\in [1,2]: \quad \\text{Bias  Terms}",font_size=20)
        SIGMOID.to_corner(UP + RIGHT).shift(DOWN * 0.6)  
        Bias.next_to(SIGMOID,DOWN).shift(DOWN * 0.05,LEFT * 0.7)
        self.add(forward_pass)
        self.forward_pass_text = forward_pass
        self.add(SIGMOID)
        self.add(Bias)
        self.SIGMOID = SIGMOID
        self.Bias = Bias
        self.wait(2)

        
        self.play(FadeOut(connections), FadeOut(weights)) # Hide connections and weights during computations
        self.play(
            LaggedStart(
                *[neuron.animate.set_fill(YELLOW, opacity=0.5) for neuron in input_layer[0]],
                lag_ratio=0.1
            )
        )
        self.wait(1)

        connections_ih_to_hidden = self.connections_ih[:len(input_layer[1]) * len(hidden_layer[1])]
        self.play(
            LaggedStart(
                *[conn.animate.set_color(YELLOW) for conn in connections_ih_to_hidden],
                lag_ratio=0.01
            )
        )

        self.wait(0.5)

        # Show computations at hidden layer
        computations = VGroup()
        for i, neuron_label in enumerate(hidden_layer[1]):
            # Display computation next to each hidden neuron
            comp = MathTex(f"=\\sigma\\left(\\sum_{{j=1}}^{{3}} w_{{{i+1}j}} x_j + b_{{{i+1}}}\\right)")
            comp.scale(0.5)
            comp.next_to(hidden_layer[0][i], RIGHT, buff=0.1)  # Increased buff
            computations.add(comp)
        self.play(Write(computations))
        self.wait(1)
        self.play(
            LaggedStart(
                *[neuron.animate.set_fill(YELLOW, opacity=0.5) for neuron in hidden_layer[0]],
                lag_ratio=0.1
            )
        )
        self.wait(1)

        # Hide computations to avoid overlap with connections
        self.play(FadeOut(computations))
        

        self.wait(0.5)

        #self.animate_wave(self.connections_oh, color=YELLOW, direction='forward',run_time=1)
        onnections_oh_to_hidden = self.connections_oh[:len(input_layer[1]) * len(hidden_layer[1])]
        self.play(
            LaggedStart(
                *[conn.animate.set_color(YELLOW) for conn in onnections_oh_to_hidden],
                lag_ratio=0.01
            )
        )

        # Show computations at output layer
        computations_out = VGroup()
        for i, neuron_label in enumerate(output_layer[1]):
            # Display computation next to each output neuron
            comp = MathTex(f"=\\sigma\\left(\\sum_{{j=1}}^{{4}} v_{{{i+1}j}} h_j + c_{{{i+1}}}\\right)")
            comp.scale(0.5)
            comp.next_to(output_layer[0][i], RIGHT, buff=0.1)  # Increased buff
            computations_out.add(comp)
        self.play(Write(computations_out))
        self.wait(1)

        # Hide computations to avoid overlap
        self.play(FadeOut(computations_out))
        
       

        # Activate output layer neurons
        self.play(
            LaggedStart(
                *[neuron.animate.set_fill(YELLOW, opacity=0.5) for neuron in output_layer[0]],
                lag_ratio=0.1
            )
        )
        self.wait(1)

        # Show connections and weights again
        self.play(FadeIn(connections), FadeIn(weights),run_time=1)
        

    def backpropagation(self, input_layer, hidden_layer, output_layer, connections, weights):
        # Remove 'Forward Pass' text
        self.play(FadeOut(self.forward_pass_text),FadeOut(self.Bias))

        # Hide connections and weights during computations
        self.play(FadeOut(connections), FadeOut(weights))

        # Add 'Backpropagation' text
        backprop_text = Text("Backpropagation", font_size=30)
        backprop_text.to_corner(DOWN + LEFT)
        COST_FUNCTION = Text("C = Cost Function", font_size=17)
        COST_FUNCTION.to_corner(RIGHT).shift(RIGHT * 0.3)
        self.add(backprop_text)
        self.backprop_text = backprop_text
        self.add(COST_FUNCTION)
        self.COST_FUNCTION = COST_FUNCTION

        # Indicate error at output layer
        self.play(
            LaggedStart(
                *[Indicate(neuron, color=RED) for neuron in output_layer[0]],
                lag_ratio=0.01
            )
        )
        self.wait(0.5)

        # Show derivative computations at output layer
        derivatives_out = VGroup()
        for i, neuron_label in enumerate(output_layer[1]):
            # Display derivative next to each output neuron
            der = MathTex(f"d_{{z_{{{i+1}}}}} =  \\frac{{\\partial C}}{{\\partial z_{{{i+1}}}}}")
            der.scale(0.5)
            der.next_to(output_layer[0][i], LEFT, buff=0.5)  # Position derivatives on the left
            derivatives_out.add(der)
        self.play(Write(derivatives_out))
        self.wait(1)

        # Hide derivative computations to avoid overlap
        self.play(FadeOut(derivatives_out))
        self.animate_wave(self.connections_oh, color=RED, direction= 'backward',run_time=1)
       

        # Activate hidden layer neurons
        self.play(
            LaggedStart(
                *[Indicate(neuron, color=RED) for neuron in hidden_layer[0]],
                lag_ratio=0.1
            )
        )
        self.wait(0.5)

        # Show derivative computations at hidden layer
        derivatives_hidden = VGroup()
        for i, neuron_label in enumerate(hidden_layer[1]):
            # Display derivative next to each hidden neuron
            der = MathTex(f"\\nu_{{{i+1},t}} = \\sum_{{k=1}}^{{2}} \\frac{{\\partial C}}{{\\partial z_{{k}}}} \\frac{{\\partial z_{{k}}}}{{\\partial v_{{{i+1},t}}}}")

            der.scale(0.5)
            der.next_to(hidden_layer[0][i], LEFT, buff=0.17)
            derivatives_hidden.add(der)
        self.play(Write(derivatives_hidden))
        self.wait(1)

        

        # Hide derivative computations to avoid overlap
        self.play(FadeOut(derivatives_hidden))
        self.animate_wave(self.connections_ih, color=RED,direction='backward',run_time=1)
        

        # Activate input layer neurons
        self.play(
            LaggedStart(
                *[Indicate(neuron, color=RED) for neuron in input_layer[0]],
                lag_ratio=0.1
            )
        )
        self.wait(0.5)

        # Show derivative computations at input layer
        derivatives_input = VGroup()
        for i, neuron_label in enumerate(input_layer[1]):

            der = MathTex(f"\\omega_{{{i+1},j}} = \\sum_{{m=1}}^{{2}} \\sum_{{k=1}}^{{4}} \\frac{{\\partial C}}{{\\partial z_{{m}}}} \\cdot \\frac{{\\partial z_{{m}}}}{{\\partial h_{{k}}}} \\frac{{\\partial h_{{k}}}}{{\\partial w_{{{i+1},j }}}}",font_size=45)

            der.scale(0.5)
            der.next_to(input_layer[0][i], LEFT, buff=0.11)
            derivatives_input.add(der)
        self.play(Write(derivatives_input))
        self.wait(1)

        # Hide derivative computations
        self.play(FadeOut(derivatives_input))
        self.play(FadeOut(self.COST_FUNCTION),FadeOut(self.SIGMOID),FadeOut(backprop_text))
        self.play(FadeOut(connections), FadeOut(weights),FadeOut(self.layers),FadeOut(self.layer_labels),FadeOut(self.SIGMOID))
        Gradient = Text("After  Gradient  Descent On  The  Weights,  We  Update  The  Weights", font_size=30)  
        Example = MathTex("\\text{For Exapmle}  : \\tilde w_{i,j} = w_{i,j} - \\epsilon \\cdot \\omega_{i,j}", font_size=35) 
        Repeat = MathTex("\\text{Repeat  This  Process  Until  The  Cost  Function  Is  Minimized}", font_size=35)
        Gradient.shift(UP * 1)
        Repeat.to_edge(DOWN)

        arrow = Arrow(LEFT,RIGHT,stroke_width=5,stroke_color=WHITE,fill_color=BLUE,fill_opacity=0.5, buff=0.1).shift(LEFT * 4.5)

        self.play(LaggedStart(FadeIn(Gradient)),run_time=1)
        self.play(LaggedStart(FadeIn(Example)),run_time=1)
        self.wait(2)
        self.play(FadeOut(Gradient),FadeOut(Example))
        self.play(FadeIn(connections.shift(RIGHT * 0.4)), FadeIn(self.weights_after.shift(RIGHT * 0.4)),
                FadeIn(self.layers.shift(RIGHT * 0.4)),FadeIn(self.layer_labels.shift(RIGHT * 0.4)),run_time=1)
        self.play(LaggedStart(FadeIn(arrow)),run_time=1)
        self.play(LaggedStart(FadeIn(Repeat)),run_time=1)

        self.wait(2)


## HMC visualization

In [ ]:
%%manim -qm -v WARNING HMCLogisticRegressionAnimation26

class HMCLogisticRegressionAnimation26(Scene):
    def construct(self):
        n_features = X.shape[1] 
        Intro_text = Text("MB", font_size=24, weight= BOLD,color= BLUE_D).to_edge(UP).shift(LEFT*6.05)
        self.add(Intro_text)
        Neural_net_text = Text("HMC sampler", font_size=18, weight= BOLD,color= BLUE_E).next_to(Intro_text,DOWN,buff=0.2).shift(RIGHT*0.1)
        self.add(Neural_net_text)
        
        # Logistic regression functions
        def sigmoid(z):
            return np.where(z >= 0, 
                            1 / (1 + np.exp(-z)), 
                            np.exp(z) / (1 + np.exp(z)))
        
        # Define log-prior, log-likelihood, and their gradients
        def log_prior(beta, tau=10):
            return -0.5 * np.sum((beta / tau) ** 2) - len(beta) * np.log(tau * np.sqrt(2 * np.pi))
        
        def log_likelihood(beta, X, y):
            linear_term = np.dot(X, beta)
            return np.sum(y * linear_term - np.log1p(np.exp(linear_term)))
        
        def log_posterior(beta, X, y, tau=10):
            return log_prior(beta, tau) + log_likelihood(beta, X, y)
        
        def gradient_log_posterior(beta, X, y, tau=10):
            linear_term = np.dot(X, beta)
            p = sigmoid(linear_term)
            grad_likelihood = np.dot(X.T, (y - p))
            grad_prior = -beta / tau**2
            return grad_likelihood + grad_prior
        
        # Hamiltonian dynamics functions remain the same
        def leapfrog_traj(beta, r, grad_log_posterior, step_size, leapfrog_steps, X, y, tau=10):
            beta_new = np.copy(beta)
            r_new = np.copy(r)
            beta_traj = [np.copy(beta_new)]
            r_new += 0.5 * step_size * grad_log_posterior(beta_new, X, y, tau)
            for _ in range(leapfrog_steps):
                beta_new += step_size * r_new
                beta_traj.append(np.copy(beta_new))
                if _ != leapfrog_steps - 1:
                    r_new += step_size * grad_log_posterior(beta_new, X, y, tau)
            r_new += 0.5 * step_size * grad_log_posterior(beta_new, X, y, tau)
            return beta_new, r_new, beta_traj
        
        # HMC parameters
        step_size = 0.045
        leapfrog_steps = 40
        num_iterations = 20
        accept_count = 0
        
        # Initial position
        beta_current = np.zeros(n_features)
        beta_samples = []
        beta_traj_samples = []
        
        # Labels
        beta_label = MathTex("\\beta \\text{ (projected)}",font_size =25).to_edge(UP).shift(LEFT*2.2,DOWN*0.4)
        self.play(Write(beta_label))
        
        # Collect initial beta_samples to fit PCA
        initial_samples = []
        num_initial_samples = 300  # Increase for better estimation
        temp_beta_current = np.copy(beta_current)
        for i in range(num_initial_samples):
            r = np.random.normal(0, 1, size=n_features)
            beta_new, r_new, beta_traj = leapfrog_traj(temp_beta_current, r, gradient_log_posterior, step_size, leapfrog_steps, X, y)
            current_H = -log_posterior(temp_beta_current, X, y) + 0.5 * np.dot(r, r)
            new_H = -log_posterior(beta_new, X, y) + 0.5 * np.dot(r_new, r_new)
            delta_H = new_H - current_H
            if np.random.uniform() < np.exp(-delta_H):
                temp_beta_current = beta_new
            initial_samples.append(temp_beta_current)
        
        
        # Fit PCA on initial_samples
        from sklearn.decomposition import PCA
        pca = PCA(n_components=2)
        pca.fit(initial_samples)
        
        # Get the ranges of the projected beta samples
        projected_samples = pca.transform(initial_samples)
        x_min, x_max = projected_samples[:, 0].min(), projected_samples[:, 0].max()
        y_min, y_max = projected_samples[:, 1].min(), projected_samples[:, 1].max()

        # Extend the ranges slightly for better visualization
        x_margin = (x_max - x_min) * 0.1
        y_margin = (y_max - y_min) * 0.1
        x_min -= x_margin
        x_max += x_margin
        y_min -= y_margin
        y_max += y_margin

        # Create axes with these ranges
        axes = Axes(
            x_range=np.round([x_min, x_max, (x_max - x_min)/5], 2),
            y_range=np.round([y_min, y_max, (y_max - y_min)/5], 2),
            x_length=7.2,
            y_length=7.2,
            tips=False
        ).add_coordinates()
        axes.center()
        self.play(Create(axes))
        
        # Create a grid over the PCA space
        xx, yy = np.meshgrid(
            np.linspace(x_min, x_max, 50),
            np.linspace(y_min, y_max, 50)
        )

        # Flatten the grid points
        grid_points = np.c_[xx.ravel(), yy.ravel()]

        # Reconstruct beta for each grid point
        beta_grid = pca.inverse_transform(grid_points)

        # Compute log_posterior for each beta in beta_grid
        log_posterior_vals = np.array([log_posterior(beta, X, y) for beta in beta_grid])

        # Reshape back to grid shape
        LOG_POSTERIOR = log_posterior_vals.reshape(xx.shape)

        # Create a matplotlib figure and axes with black background
        fig, ax = plt.subplots(figsize=(6, 6))

        # Set the background color of the figure and axes to black
        fig.patch.set_facecolor('black')
        ax.set_facecolor('black')

        # Plot the filled contour
        max_log_posterior = np.max(LOG_POSTERIOR)
        min_log_posterior = np.min(LOG_POSTERIOR)
        contour_levels = np.linspace(min_log_posterior, max_log_posterior, 15)

        # Use a colormap that stands out against a black background
        cmap = plt.cm.plasma  # You can choose 'viridis', 'inferno', 'magma', etc.

        # Plot the filled contour
        CS = ax.contourf(xx, yy, LOG_POSTERIOR, levels=contour_levels, cmap=cmap)

        # Optional: Add contour lines for reference, in a color that stands out
        ax.contour(xx, yy, LOG_POSTERIOR, levels=contour_levels, colors='white', linewidths=0.5)

        # Remove axis labels and ticks for a cleaner look
        ax.axis('off')

        # Adjust colorbar with white labels and black background
        #cbar = fig.colorbar(CS, ax=ax)
        #cbar.ax.yaxis.set_tick_params(color='white')
        #plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='white')
        #cbar.outline.set_edgecolor('white')
        #cbar.ax.set_facecolor('black')

        # Convert the plot to an image that Manim can display
        fig.canvas.draw()
        contour_image = ImageMobject(np.array(fig.canvas.renderer.buffer_rgba()))
        contour_image.scale_to_fit_width(axes.width)
        contour_image.move_to(axes)

        plt.close(fig)  # Close the figure to free memory

        # Display the filled contour in Manim
        self.add(contour_image)
        self.bring_to_back(contour_image)


        
        # Initialize the current_dot
        current_proj = pca.transform([beta_current])[0]
        current_point = axes.coords_to_point(current_proj[0], current_proj[1])
        current_dot = Dot(current_point, color=BLUE)
        self.play(FadeIn(current_dot))
        
        for i in range(num_iterations):
            r = np.random.normal(0, 1, size=n_features)
            start_beta = np.copy(beta_current)
            beta_new, r_new, beta_traj = leapfrog_traj(beta_current, r, gradient_log_posterior, step_size, leapfrog_steps, X, y)
            
            # Hamiltonian at current and proposed positions
            current_H = -log_posterior(beta_current, X, y) + 0.5 * np.dot(r, r)
            new_H = -log_posterior(beta_new, X, y) + 0.5 * np.dot(r_new, r_new)
            delta_H = new_H - current_H
            
            # Metropolis acceptance criterion
            if np.random.uniform() < np.exp(-delta_H):
                beta_current = beta_new
                accept_count += 1
                accepted = True
            else:
                beta_current = start_beta
                accepted = False
            
            beta_samples.append(beta_current)
            beta_traj_samples.extend(beta_traj)
            
            # Project the trajectory using fixed PCA components
            projected_traj = pca.transform(beta_traj)
            
            # Convert projected points to Manim coordinates
            traj_points = [axes.coords_to_point(pt[0], pt[1]) for pt in projected_traj]
            traj = VMobject()
            traj.set_points_smoothly(traj_points)
            traj.set_stroke(color=PURPLE)
            
            # Animate the dot moving along the trajectory
            self.play(MoveAlongPath(current_dot, traj), Create(traj), run_time=0.9)
            self.play(Transform(traj, traj.copy().set_stroke(color=PURPLE, width=0.15)))
            
            # Indicate acceptance or rejection
            if accepted:
                self.play(Indicate(current_dot, color=GREEN, scale_factor=1.2))
            else:
                self.play(Indicate(current_dot, color=RED, scale_factor=1.2))
                # Move the dot back to the start position
                start_proj = pca.transform([start_beta])[0]
                start_point = axes.coords_to_point(start_proj[0], start_proj[1])
                self.play(current_dot.animate.move_to(start_point))
        
        # Display acceptance rate
        acceptance_rate = accept_count / num_iterations
        acceptance_text = Text(f"Acceptance Rate: {acceptance_rate:.2f}", font_size=24).to_edge(RIGHT).shift(RIGHT*0.38)
        self.play(Write(acceptance_text))
        self.wait(2)
